In [5]:
!pip install boto3

In [6]:
import requests
import boto3
import os
import random
from datetime import datetime, timezone, timedelta

In [45]:
LABEL_STUDIO_URL="http://label-studio:8080"
LABEL_STUDIO_TOKEN="ab9927067c51ff279d340d7321e4890dc2841c4a"
MINIO_URL="http://minio:9000"
MINIO_ACCESS_KEY ="PROSadmin19"
MINIO_SECRET_KEY = MINIO_ROOT_PASSWORD="PROSadmin19"
BUCKET_NAME = "production"
PUBLIC_IP = "129.114.26.168"

In [63]:
# Connect project to MinIO storage
storage_config = {
    # "project": PROJECT_ID,
    "type": "s3",
    "title": "MinIO",
    "bucket": BUCKET_NAME,
    "aws_access_key_id": MINIO_ACCESS_KEY,
    "aws_secret_access_key": MINIO_SECRET_KEY,
    "endpoint_url": f"http://{PUBLIC_IP}:9000",  # Using public IP
    "recursive_scan": True,
    "treat_every_object_as_file": True
}

storage_config

{'type': 's3',
 'title': 'MinIO',
 'bucket': 'production',
 'aws_access_key_id': 'PROSadmin19',
 'aws_secret_access_key': 'PROSadmin19',
 'endpoint_url': 'http://129.114.26.168:9000',
 'recursive_scan': True,
 'treat_every_object_as_file': True}

In [ ]:
# Labeling interface configuration
LABEL_CONFIG = """
<View>
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse"><Image name="image" value="$image" maxWidth="800px" zoom="true" zoomControl="true"/><RectangleLabels name="label" toName="image" showInline="true">
    <Label value="0" background="#FFA39E"/>
    <Label value="1" background="#D4380D"/>
    <Label value="2" background="#FFC069"/>
    <Label value="3" background="#AD8B00"/>
    <Label value="4" background="#D3F261"/>
    <Label value="5" background="#389E0D"/>
    <Label value="6" background="#5CDBD3"/>
    <Label value="7" background="#096DD9"/>
    <Label value="8" background="#ADC6FF"/>
    <Label value="9" background="#9254DE"/>
    <Label value="10" background="#F759AB"/>
    <Label value="11" background="#FFA39E"/>
    <Label value="12" background="#D4380D"/>
    <Label value="13" background="#FFC069"/>
  </RectangleLabels></View>
  <Header value="Model Confidence: $confidence"/>
  <Header value="Predicted Class: $predicted_class"/>
  
</View>
"""

In [64]:
# Get the public IP of your VM (needed for pre-signed URLs)
public_ip = PUBLIC_IP

# Initialize MinIO client
s3 = boto3.client(
    "s3",
    endpoint_url=f"http://{public_ip}:9000",  # Use public IP for external access
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY,
    region_name="us-east-1"
)

# Create Project - Low Confidence

In [49]:
# Create the Label Studio project
headers = {"Authorization": f"Token {LABEL_STUDIO_TOKEN}"}
project_config = {
    "title": "Chest X-Ray Low Confidence",
    "description": "Label chest X-ray images where the model has low confidence",
    "label_config": LABEL_CONFIG
}

res = requests.post(f"{LABEL_STUDIO_URL}/api/projects", json=project_config, headers=headers)
if res.status_code == 201:
    LC_PROJECT_ID = res.json()['id']
    print(f"Created project: Chest X-Ray Low Confidence (ID {LC_PROJECT_ID})")
else:
    raise Exception(f"Failed to create project: {res.text}")

Created project: Chest X-Ray Low Confidence (ID 2)


## Add Low Confidence Images

In [66]:
# Find low-confidence images
low_confidence_threshold = 0.95  # Images with confidence below this
recent_time_threshold = datetime.now(timezone.utc) - timedelta(hours=12)  # Images from last 12 hours
all_keys = []

paginator = s3.get_paginator("list_objects_v2")
for page in paginator.paginate(Bucket=BUCKET_NAME):
    for obj in page.get("Contents", []):
        key = obj["Key"]
        tags = s3.get_object_tagging(Bucket=BUCKET_NAME, Key=key)['TagSet']
        tag_dict = {t['Key']: t['Value'] for t in tags}
        timestamp_str = tag_dict.get("timestamp")
        confidence_str = tag_dict.get("confidence")
        predicted_class = tag_dict.get("predicted_class", "")
        if timestamp_str and confidence_str:
            ts = datetime.fromisoformat(timestamp_str)
            confidence = float(confidence_str)
            if ts > recent_time_threshold and confidence < low_confidence_threshold:
                all_keys.append({
                    "key": key,
                    "confidence": confidence_str,
                    "predicted_class": predicted_class
                })

# Check if we found any low-confidence images
print(f"Found {len(all_keys)} low-confidence images:")
for item in all_keys:
    print(item)

# Create Label Studio tasks
headers = {"Authorization": f"Token {LABEL_STUDIO_TOKEN}"}
tasks = []
for item in all_keys:
    key = item["key"]
    confidence = item["confidence"]
    predicted_class = item["predicted_class"]
    presigned_url = s3.generate_presigned_url(
        'get_object',
        Params={'Bucket': BUCKET_NAME, 'Key': key},
        ExpiresIn=3600  # URL valid for 1 hour
    )
    tasks.append({
        "data": {
            "image": presigned_url,
            "confidence": confidence,
            "predicted_class": predicted_class
        },
        "meta": {"original_key": key}
    })

# Send tasks to Label Studio
res = requests.post(
    f"{LABEL_STUDIO_URL}/api/projects/{LC_PROJECT_ID}/import",
    json=tasks,
    headers=headers
)
if res.status_code == 201:
    print(f"Imported {len(tasks)} tasks into project {LC_PROJECT_ID}")
else:
    print(f"Failed to import tasks: {res.text}")

Found 9 low-confidence images:
{'key': 'class_class2/0c5312c5-e3da-41de-9a33-12d503142cd5.jpg', 'confidence': '0.938', 'predicted_class': 'class2'}
{'key': 'class_class2/247ff5b0-94ce-48ea-b991-263ec6c45cef.jpg', 'confidence': '0.938', 'predicted_class': 'class2'}
{'key': 'class_class2/30f525b0-6db8-4a6f-83a3-2153138d8730.jpg', 'confidence': '0.938', 'predicted_class': 'class2'}
{'key': 'class_class2/3be42a82-7b51-4d74-a883-ddaba216ec86.jpg', 'confidence': '0.938', 'predicted_class': 'class2'}
{'key': 'class_class2/80dd93d9-d009-4db0-9a11-2cb713233ac0.jpg', 'confidence': '0.938', 'predicted_class': 'class2'}
{'key': 'class_class2/82838353-2bfa-4530-afad-94af3c0a9697.jpg', 'confidence': '0.938', 'predicted_class': 'class2'}
{'key': 'class_class2/92506c44-3918-43e7-ab93-719b4baa9b19.jpg', 'confidence': '0.938', 'predicted_class': 'class2'}
{'key': 'class_class2/b780b884-4d6b-449c-9b6b-f4efe3f6dfce.jpg', 'confidence': '0.938', 'predicted_class': 'class2'}
{'key': 'class_class2/c6e30db9-02

# Create Project 2 - Feedback Images

In [69]:
# Create the Label Studio project
headers = {"Authorization": f"Token {LABEL_STUDIO_TOKEN}"}
project_config = {
    "title": "Chest X-Ray Fedback Images",
    "description": "Label chest X-ray images where the model has low confidence",
    "label_config": LABEL_CONFIG
}

res = requests.post(f"{LABEL_STUDIO_URL}/api/projects", json=project_config, headers=headers)
if res.status_code == 201:
    FI_PROJECT_ID = res.json()['id']
    print(f"Created project: Chest X-Ray Fedback Images (ID {FI_PROJECT_ID})")
else:
    raise Exception(f"Failed to create project: {res.text}")

Created project: Chest X-Ray Fedback Images (ID 3)


## Add Feedback Images

In [70]:
# Find flagged images
recent_time_threshold = datetime.now(timezone.utc) - timedelta(hours=12)
all_keys = []

paginator = s3.get_paginator("list_objects_v2")
for page in paginator.paginate(Bucket=BUCKET_NAME):
    for obj in page.get("Contents", []):
        key = obj["Key"]
        tags = s3.get_object_tagging(Bucket=BUCKET_NAME, Key=key)['TagSet']
        tag_dict = {t['Key']: t['Value'] for t in tags}
        timestamp_str = tag_dict.get("timestamp")
        flagged = tag_dict.get("flagged") == "true"
        if timestamp_str and flagged:
            ts = datetime.fromisoformat(timestamp_str)
            if ts > recent_time_threshold:
                all_keys.append({
                    "key": key,
                    "confidence": tag_dict.get("confidence", ""),
                    "predicted_class": tag_dict.get("predicted_class", ""),
                    "flagged": tag_dict.get("flagged", "false")
                })

print(f"Found {len(all_keys)} flagged images:")
for item in all_keys:
    print(item)

# Create Label Studio tasks
headers = {"Authorization": f"Token {LABEL_STUDIO_TOKEN}"}
tasks = []
for item in all_keys:
    key = item["key"]
    confidence = item["confidence"]
    predicted_class = item["predicted_class"]
    presigned_url = s3.generate_presigned_url(
        'get_object',
        Params={'Bucket': BUCKET_NAME, 'Key': key},
        ExpiresIn=3600
    )
    tasks.append({
        "data": {
            "image": presigned_url,
            "confidence": confidence,
            "predicted_class": predicted_class,
            "flagged": item["flagged"]
        },
        "meta": {"original_key": key}
    })

res = requests.post(
    f"{LABEL_STUDIO_URL}/api/projects/{FI_PROJECT_ID}/import",
    json=tasks,
    headers=headers
)
if res.status_code == 201:
    print(f"Imported {len(tasks)} tasks into project {FI_PROJECT_ID}")
else:
    print(f"Failed to import tasks: {res.text}")

Found 5 flagged images:
{'key': 'class_class2/0c82a178-1e9c-4ad6-9adc-bdf350057e8f.jpg', 'confidence': '0.938', 'predicted_class': 'class2', 'flagged': 'true'}
{'key': 'class_class2/1e58ba88-a5bd-4105-a66f-e8ed81e34712.jpg', 'confidence': '0.938', 'predicted_class': 'class2', 'flagged': 'true'}
{'key': 'class_class2/2c21d280-e47a-4a44-bfcd-06d39ecf64d2.jpg', 'confidence': '0.938', 'predicted_class': 'class2', 'flagged': 'true'}
{'key': 'class_class2/87e4dac7-d5a2-483a-b78b-b43daedd3c0e.jpg', 'confidence': '0.938', 'predicted_class': 'class2', 'flagged': 'true'}
{'key': 'class_class2/df2d0522-b588-44d5-a254-f1e16ebfb3cf.jpg', 'confidence': '0.938', 'predicted_class': 'class2', 'flagged': 'true'}
Imported 5 tasks into project 3


# Evaluate Performance - Low Confidence

In [ ]:
# Get human labels from Label Studio
response = requests.get(
    f"{LABEL_STUDIO_URL}/api/projects/{LC_PROJECT_ID}/export?exportType=JSON",
    headers={"Authorization": f"Token {LABEL_STUDIO_TOKEN}"}
)
tasks = response.json()

# Compare with model predictions
total, correct = 0, 0
for task in tasks:
    human_label = task['annotations'][0]['result'][0]['value']['choices'][0]
    key = task['meta']['original_key']
    tags = s3.get_object_tagging(Bucket=BUCKET_NAME, Key=key)['TagSet']
    model_label = {t['Key']: t['Value'] for t in tags}.get('predicted_class')
    if model_label and human_label:
        total += 1
        correct += int(model_label == human_label)

print(f"Accuracy: {correct}/{total} = {correct / total:.2%}" if total else "No valid comparisons.")

# Evaluate Performance - Flagged Images

In [71]:
# Get human labels from Label Studio
response = requests.get(
    f"{LABEL_STUDIO_URL}/api/projects/{FI_PROJECT_ID}/export?exportType=JSON",
    headers={"Authorization": f"Token {LABEL_STUDIO_TOKEN}"}
)
tasks = response.json()

# Compare with model predictions
total, correct = 0, 0
for task in tasks:
    human_label = task['annotations'][0]['result'][0]['value']['choices'][0]
    key = task['meta']['original_key']
    tags = s3.get_object_tagging(Bucket=BUCKET_NAME, Key=key)['TagSet']
    model_label = {t['Key']: t['Value'] for t in tags}.get('predicted_class')
    if model_label and human_label:
        total += 1
        correct += int(model_label == human_label)

print(f"Accuracy: {correct}/{total} = {correct / total:.2%}" if total else "No valid comparisons.")

No valid comparisons.
